In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/airfoil-selfnoise-dataset/AirfoilSelfNoise.csv


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=481cc74913ff82988de14c971a17516aa705814e5b5a277380fef1d81fe6cf56
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [5]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 13:25:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

In [7]:
df = spark.read.csv("/kaggle/input/airfoil-selfnoise-dataset",header=True,inferSchema='True')

In [8]:
df.printSchema()

root
 |-- f: integer (nullable = true)
 |-- alpha: double (nullable = true)
 |-- c: double (nullable = true)
 |-- U_infinity: double (nullable = true)
 |-- delta: double (nullable = true)
 |-- SSPL: double (nullable = true)



In [9]:
df.show(2)

+----+-----+------+----------+----------+-------+
|   f|alpha|     c|U_infinity|     delta|   SSPL|
+----+-----+------+----------+----------+-------+
| 800|  0.0|0.3048|      71.3|0.00266337|126.201|
|1000|  0.0|0.3048|      71.3|0.00266337|125.201|
+----+-----+------+----------+----------+-------+
only showing top 2 rows



In [10]:
#train and test split
train,test = df.randomSplit([0.7,0.3])

In [11]:
print("train",train.count())
print("test",test.count())

train 1057
test 446


In [12]:
train.show(2)
test.show(2)

+---+-----+------+----------+---------+-------+
|  f|alpha|     c|U_infinity|    delta|   SSPL|
+---+-----+------+----------+---------+-------+
|200|  7.3|0.2286|      31.7|0.0132672|128.679|
|200|  7.3|0.2286|      39.6|0.0123481|130.989|
+---+-----+------+----------+---------+-------+
only showing top 2 rows

+---+-----+------+----------+----------+-------+
|  f|alpha|     c|U_infinity|     delta|   SSPL|
+---+-----+------+----------+----------+-------+
|200|  0.0|0.3048|      31.7|0.00331266|117.195|
|200|  0.0|0.3048|      39.6|0.00310138|118.129|
+---+-----+------+----------+----------+-------+
only showing top 2 rows



In [13]:
#Identifying the categorical and numerical columns
cat_cols = [x for x, dataType in train.dtypes if dataType=="String"]
num_cols = [x for x, dataType in train.dtypes if dataType!="String"]

In [14]:
print(cat_cols)

[]


In [15]:
print(num_cols)

['f', 'alpha', 'c', 'U_infinity', 'delta', 'SSPL']


In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
assembler_input = [x for x in num_cols]

In [18]:
assembler_input

['f', 'alpha', 'c', 'U_infinity', 'delta', 'SSPL']

In [19]:
vector_assembler = VectorAssembler(inputCols=assembler_input, outputCol="VectorAssemblerFeature")

In [20]:
stages=[]
stages += [vector_assembler]

In [21]:
stages

[VectorAssembler_d4561d045f9c]

In [22]:
%%time
from pyspark.ml import Pipeline

pipeline = Pipeline().setStages(stages)
model = pipeline.fit(train)
transformed = model.transform(test)


CPU times: user 2.46 ms, sys: 2.13 ms, total: 4.58 ms
Wall time: 121 ms


In [23]:
pp_df = model._transform(test)

In [24]:
pp_df.select('f', 'alpha', 'c', 'U_infinity', 'delta', 'SSPL',"VectorAssemblerFeature")

DataFrame[f: int, alpha: double, c: double, U_infinity: double, delta: double, SSPL: double, VectorAssemblerFeature: vector]

In [25]:
pp_df.show(truncate=False)

+---+-----+------+----------+----------+-------+------------------------------------------+
|f  |alpha|c     |U_infinity|delta     |SSPL   |VectorAssemblerFeature                    |
+---+-----+------+----------+----------+-------+------------------------------------------+
|200|0.0  |0.3048|31.7      |0.00331266|117.195|[200.0,0.0,0.3048,31.7,0.00331266,117.195]|
|200|0.0  |0.3048|39.6      |0.00310138|118.129|[200.0,0.0,0.3048,39.6,0.00310138,118.129]|
|200|8.9  |0.1016|39.6      |0.0124596 |133.42 |[200.0,8.9,0.1016,39.6,0.0124596,133.42]  |
|200|9.9  |0.1524|71.3      |0.0193001 |134.319|[200.0,9.9,0.1524,71.3,0.0193001,134.319] |
|200|12.3 |0.1016|31.7      |0.0418756 |124.987|[200.0,12.3,0.1016,31.7,0.0418756,124.987]|
|200|12.3 |0.1016|39.6      |0.0408268 |128.545|[200.0,12.3,0.1016,39.6,0.0408268,128.545]|
|200|12.3 |0.1016|71.3      |0.0337792 |130.588|[200.0,12.3,0.1016,71.3,0.0337792,130.588]|
|200|15.6 |0.1016|39.6      |0.0528487 |123.514|[200.0,15.6,0.1016,39.6,0.052848

In [26]:
from pyspark.ml.regression import LinearRegression

In [27]:
data = pp_df.select(F.col("VectorAssemblerFeature").alias("features"),
                    F.col("alpha").alias("label"),
                    )

In [28]:
data.show(5,truncate=False)

+------------------------------------------+-----+
|features                                  |label|
+------------------------------------------+-----+
|[200.0,0.0,0.3048,31.7,0.00331266,117.195]|0.0  |
|[200.0,0.0,0.3048,39.6,0.00310138,118.129]|0.0  |
|[200.0,8.9,0.1016,39.6,0.0124596,133.42]  |8.9  |
|[200.0,9.9,0.1524,71.3,0.0193001,134.319] |9.9  |
|[200.0,12.3,0.1016,31.7,0.0418756,124.987]|12.3 |
+------------------------------------------+-----+
only showing top 5 rows



In [29]:
%%time
model = LinearRegression().fit(data)

23/11/20 13:25:39 WARN Instrumentation: [9a9be97b] regParam is zero, which might cause numerical instability and overfitting.
23/11/20 13:25:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/20 13:25:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


CPU times: user 14.2 ms, sys: 5.87 ms, total: 20.1 ms
Wall time: 1.37 s


In [31]:
print("MAE",model.summary.meanAbsoluteError)

MAE 2.555285314404431e-14


In [32]:
print("MSE",model.summary.meanSquaredError)

MSE 1.1055119372727936e-27


In [33]:
print("r2 score",model.summary.r2)

r2 score 1.0


In [34]:
print("RMSE",model.summary.rootMeanSquaredError)

RMSE 3.3249239649543773e-14


In [35]:
#save the model
model.save("linear model")

In [36]:
#load the model
loded_model = model.load("linear model")